# Initial Setup

Import statements

In [ ]:
from classifier.network.toy_net import *
from classifier.network.alex_net import *
from classifier.plugin import *
from classifier.metric import *
from preprocess import *

Paths definition


In [ ]:
DATASET_PATH = Path("../dataset")
SUBMISSION_PATH = Path("submissions")
TRAINED_MODELS_PATH = Path("trained-models")
PROCESSED_DATA_PATH = Path("processed-data")
WRONG_PRED_ENTRIES_PATH = Path("wrong-pred-entries")

Optimizer setup

In [ ]:
ADAM_PROFILE = OptimizerProfile(Adam, {
            "lr": 0.0005,
            "betas": (0.9, 0.99),
            "eps": 1e-8
        })

SGD_PROFILE = OptimizerProfile(SGD, {
        'lr': 0.0005,
        'momentum': 0.99
})

# A toy demo

In [ ]:
trained_toy_net_PATH = Path(TRAINED_MODELS_PATH / "toy-net")
def toy_demo():
    training_set = read_train_labeled(DATASET_PATH)
    validation_proportion = 0.1
    validation, train = partition(training_set, [int(len(training_set) * validation_proportion)], True)
    toy_clf = ToyClassifier(train, validation)
    toy_clf.train(epochs=100,
                  batch_size=100,
                  plugins=[
                      #save_model(trained_toy_net_PATH),
                      #calc_train_val_performance(accuracy),
                      print_train_val_performance(accuracy),
                      #log_train_val_performance(accuracy),
                      #save_training_message(trained_toy_net_PATH),
                      #save_train_val_performance(trained_toy_net_PATH, accuracy),
                      plot_train_val_performance(trained_toy_net_PATH, 'Modified AlexNet', accuracy, True, True)
                  ]
                  )
    #toy_clf = ToyClassifier(train, validation)
    #toy_clf.load_network(trained_toy_net_PATH, 2)
    print(toy_clf.val_performance(accuracy))


In [ ]:
toy_demo()

# Model Selection

In [ ]:
# path for storing augmented data
rotation_augmented_data_PATH = Path(PROCESSED_DATA_PATH / "rotation-augmented-dataset-last90percent.data")

# path for storing augmented data, images containing x or t are not rotated
rotation_augmented_ignorext_data_PATH = Path(PROCESSED_DATA_PATH / "rotation-augmented-ignorext-dataset-last90percent.data")

# path for all alex nets
TRAINED_ALEX_NET_PATH = Path(TRAINED_MODELS_PATH / "alex-nets")

# path for storing wrong classified images
ALEX_NET_WRONG_PREDICTION_PATH = Path(WRONG_PRED_ENTRIES_PATH / "alex-nets")

In [ ]:
def run_alex(n_way: int, depth: Tuple[int, int, int], scaled: bool, relabeled: bool, rotation_augment = False, ignorext = False):
    print(f"n_way: {n_way}")
    print(f"depth: {depth}")
    training_set = read_train_labeled(DATASET_PATH)
    validation_proportion = 0.1
    trained_alex_net_PATH = TRAINED_ALEX_NET_PATH
    alex_net_rotation_augmented_WRONG_PRED_PATH = ALEX_NET_WRONG_PREDICTION_PATH
    if rotation_augment:
        trained_alex_net_PATH = Path(Path(str(trained_alex_net_PATH) + "-rotation-augmented"))
        alex_net_rotation_augmented_WRONG_PRED_PATH = Path(Path(str(alex_net_rotation_augmented_WRONG_PRED_PATH) + "-rotation-augmented"))
        if ignorext:
            trained_alex_net_PATH = Path(Path(str(trained_alex_net_PATH) + "-rotation-augmented-ignorext"))
            alex_net_rotation_augmented_WRONG_PRED_PATH = Path(
                Path(str(alex_net_rotation_augmented_WRONG_PRED_PATH) + "-rotation-augmented-ignorext"))

        validation, train = partition(training_set, [int(len(training_set) * validation_proportion)], False)
        train = torch.load(rotation_augmented_data_PATH)
    else:
        validation, train = partition(training_set, [int(len(training_set) * validation_proportion)], False)
    if scaled:
        trained_alex_net_PATH = Path(Path(str(trained_alex_net_PATH) + "-scaled"))
        alex_net_rotation_augmented_WRONG_PRED_PATH = Path(
            Path(str(alex_net_rotation_augmented_WRONG_PRED_PATH) + "-scaled"))
        train = preprocess_scale_image(train)
        validation = preprocess_scale_image(validation)
    if relabeled:
        trained_alex_net_PATH = Path(Path(Path(str(trained_alex_net_PATH) + "-relabeled")))
        alex_net_rotation_augmented_WRONG_PRED_PATH = Path(
            Path(str(alex_net_rotation_augmented_WRONG_PRED_PATH) + "-relabeled"))
        train = preprocess_260_labels(train)
        validation = preprocess_260_labels(validation)

    trained_alex_net_PATH = Path(trained_alex_net_PATH / f"{n_way}way-depth{depth}")
    alex_net_rotation_augmented_WRONG_PRED_PATH = Path(alex_net_rotation_augmented_WRONG_PRED_PATH / f"{n_way}way-depth{depth}")
    alex = AlexNetClassifier(train, validation, n_way=n_way, depth=depth, scaled=scaled, relabeled=relabeled)
    alex.set_optimizer(ADAM_PROFILE)
    alex.train(30,
               batch_size=100,
               plugins=[
                   save_model(trained_alex_net_PATH),
                   calc_train_val_performance(accuracy),
                   print_train_val_performance(accuracy),
                   log_train_val_performance(accuracy),
                   save_training_message(trained_alex_net_PATH),
                   plot_train_val_performance(trained_alex_net_PATH, 'Modified AlexNet', accuracy, show=False, save=True),
                   elapsed_time(),
                   save_train_val_performance(trained_alex_net_PATH, accuracy),
               ])
    alex.extract_wrong_pred_entries(alex_net_rotation_augmented_WRONG_PRED_PATH)

Run 2-way networks with different depth of feature extraction in each scale space

In [ ]:
run_alex(2, (1,1,2), scaled=False, relabeled=False, rotation_augment=False) # original AlexNet
run_alex(2, (1,1,3), scaled=False, relabeled=False, rotation_augment=False)
run_alex(2, (1,2,2), scaled=False, relabeled=False, rotation_augment=False)
run_alex(2, (1,2,3), scaled=False, relabeled=False, rotation_augment=False)
run_alex(2, (2,2,4), scaled=False, relabeled=False, rotation_augment=False)
run_alex(2, (2,3,3), scaled=False, relabeled=False, rotation_augment=False)
run_alex(2, (2,3,4), scaled=False, relabeled=False, rotation_augment=False)
run_alex(2, (3,3,5), scaled=False, relabeled=False, rotation_augment=False)
run_alex(2, (3,4,4), scaled=False, relabeled=False, rotation_augment=False)
run_alex(2, (3,4,5), scaled=False, relabeled=False, rotation_augment=False)

We tested (3,4,4) to be the best. Now try different number of ways

In [ ]:
run_alex(1, (3,4,4), scaled=False, relabeled=False, rotation_augment=False)
run_alex(3, (3,4,4), scaled=False, relabeled=False, rotation_augment=False)
run_alex(4, (3,4,4), scaled=False, relabeled=False, rotation_augment=False)

The 1-way model performs almost as well as 2-way. Now confirm that the depth is still the best in 1-way models

In [ ]:
run_alex(1, (1,1,2), scaled=False, relabeled=False, rotation_augment=False)
run_alex(1, (1,1,3), scaled=False, relabeled=False, rotation_augment=False)
run_alex(1, (1,2,2), scaled=False, relabeled=False, rotation_augment=False)
run_alex(1, (1,2,3), scaled=False, relabeled=False, rotation_augment=False)
run_alex(1, (2,2,4), scaled=False, relabeled=False, rotation_augment=False)
run_alex(1, (2,3,3), scaled=False, relabeled=False, rotation_augment=False)
run_alex(1, (2,3,4), scaled=False, relabeled=False, rotation_augment=False)
run_alex(1, (3,3,5), scaled=False, relabeled=False, rotation_augment=False)
run_alex(1, (3,4,4), scaled=False, relabeled=False, rotation_augment=False)
run_alex(1, (3,4,5), scaled=False, relabeled=False, rotation_augment=False)

We use the 1 way version with depths (3,4,4) as our best single model.

# Preprocesses

Test impact of scaling the image to 112*112 and/or expand the labels of the dataset from 34 zeros with 2 ones to 260 labels via a surjective mapping.

In [ ]:
run_alex(1, (3,4,4), scaled=False, relabeled=False, rotation_augment=False)
run_alex(1, (3,4,4), scaled=True, relabeled=False, rotation_augment=False)
run_alex(1, (3,4,4), scaled=False, relabeled=True, rotation_augment=False)
run_alex(1, (3,4,4), scaled=True, relabeled=True, rotation_augment=False)

Test impact of augmenting the training data by rotations of +/- 10 20 30 degrees.

In [ ]:
# save the preprocessed data
validation, train = partition(training_set, [int(len(training_set) * validation_proportion)], False)
process_data(train, rotation_augmented_data_PATH, [preprocess_rotate,],
                     [{'rotations': [-30, -20, -10, 0, 10, 20, 30]}]
                    )

In [ ]:
run_alex(1, (3,4,4), scaled=False, relabeled=False, rotation_augment=True)
run_alex(1, (3,4,4), scaled=True, relabeled=False, rotation_augment=True)
run_alex(1, (3,4,4), scaled=False, relabeled=True, rotation_augment=True)
run_alex(1, (3,4,4), scaled=True, relabeled=True, rotation_augment=True)

# Semisupervised Learning

In [ ]:
run_alex(1, (3, 4, 4), scaled=False, relabeled=False, rotation_augment=False, semisupervised=True)
run_alex(1, (3, 4, 4), scaled=True, relabeled=False, rotation_augment=False, semisupervised=True)
run_alex(1, (3, 4, 4), scaled=False, relabeled=True, rotation_augment=False, semisupervised=True)
run_alex(1, (3, 4, 4), scaled=True, relabeled=True, rotation_augment=False, semisupervised=True)

# Committee of Classifiers
A group of trained classifiers is chosen, and the result is obtained by polling with weights assigned to each classifier.

Our best result comes from a committe of 8 classifiers. 4 trained on augmented dataset, 4 trained using semisupervised learning.

In [ ]:
# obtain validation accuracy
def run_committee_val():
    training_set = read_train_labeled(DATASET_PATH)
    validation_proportion = 0.1
    validation, train = partition(training_set, [int(len(training_set) * validation_proportion)], False)
    train_scaled = preprocess_scale_image(train)
    train_scaled_relabeled = preprocess_260_labels(train_scaled)
    train_relabeled = preprocess_260_labels(train)

    validation_scaled = preprocess_scale_image(validation)
    validation_scaled_relabeled = preprocess_260_labels(validation_scaled)
    validation_relabeled = preprocess_260_labels(validation)

    p1 = {'n_way': 1, 'depth': (3, 4, 4), 'scaled': False, 'relabeled': False}
    p2 = {'n_way': 1, 'depth': (3, 4, 4), 'scaled': True, 'relabeled': False}
    p3 = {'n_way': 1, 'depth': (3, 4, 4), 'scaled': False, 'relabeled': True}
    p4 = {'n_way': 1, 'depth': (3, 4, 4), 'scaled': True, 'relabeled': True}

    original = Path(TRAINED_MODELS_PATH / 'alex-nets-rotation-augmented' / '1way-depth(3, 4, 4)')
    scaled = Path(TRAINED_MODELS_PATH / 'alex-nets-rotation-augmented-scaled' / '1way-depth(3, 4, 4)')
    relabeled = Path(TRAINED_MODELS_PATH / 'alex-nets-rotation-augmented-relabeled' / '1way-depth(3, 4, 4)')
    scaled_relabeled = Path(
        TRAINED_MODELS_PATH / 'alex-nets-rotation-augmented-scaled-relabeled' / '1way-depth(3, 4, 4)')

    alex1 = AlexNetClassifier(train, validation, **p1)
    alex2 = AlexNetClassifier(train_scaled, validation_scaled, **p2)
    alex3 = AlexNetClassifier(train_relabeled, validation_relabeled, **p3)
    alex4 = AlexNetClassifier(train_scaled_relabeled, validation_scaled_relabeled, **p4)
    alex1.load_network(original, 28)
    alex2.load_network(scaled, 30)
    alex3.load_network(relabeled, 7)
    alex4.load_network(scaled_relabeled, 28)
    pred1 = Tensor([]).to(alex1.device)
    pred2 = Tensor([]).to(alex2.device)
    pred3 = Tensor([]).to(alex3.device)
    pred4 = Tensor([]).to(alex4.device)
    true = Tensor([]).to(alex1.device)
    loader = DataLoader(validation, batch_size=500, shuffle=False)
    for i, data in enumerate(loader, 0):
        x = data[0].to(alex1.device)
        y = data[1].to(alex1.device)
        pred1 = torch.cat((pred1, Function.label_to_36_argmax(alex1.predict(x), device=alex1.device)), dim=0)
        pred3 = torch.cat((pred3, Function.label_to_36_argmax(alex3.predict(x), device=alex3.device)), dim=0)
        true = torch.cat((true, y), dim=0)

    loader = DataLoader(validation_scaled, batch_size=500, shuffle=False)
    for i, data in enumerate(loader, 0):
        x = data[0].to(alex1.device)
        pred2 = torch.cat((pred2, Function.label_to_36_argmax(alex2.predict(x), device=alex1.device)), dim=0)
        pred4 = torch.cat((pred4, Function.label_to_36_argmax(alex4.predict(x), device=alex3.device)), dim=0)
    pred = pred1 * 1.2 + pred2 * 1.1 + pred3 + pred4

    original = Path(TRAINED_MODELS_PATH / 'alex-nets' / '1way-depth(3, 4, 4)-fixmatch')
    scaled = Path(TRAINED_MODELS_PATH / 'alex-nets-scaled' / '1way-depth(3, 4, 4)-fixmatch')
    relabeled = Path(TRAINED_MODELS_PATH / 'alex-nets-relabeled' / '1way-depth(3, 4, 4)-fixmatch')
    scaled_relabeled = Path(
        TRAINED_MODELS_PATH / 'alex-nets-scaled-relabeled' / '1way-depth(3, 4, 4)-fixmatch')

    alex1 = AlexNetClassifier(train, validation, **p1)
    alex2 = AlexNetClassifier(train_scaled, validation_scaled, **p2)
    alex3 = AlexNetClassifier(train_relabeled, validation_relabeled, **p3)
    alex4 = AlexNetClassifier(train_scaled_relabeled, validation_scaled_relabeled, **p4)
    alex1.load_network(original, 26)
    alex2.load_network(scaled, 24)
    alex3.load_network(relabeled, 29)
    alex4.load_network(scaled_relabeled, 29)
    pred1 = Tensor([]).to(alex1.device)
    pred2 = Tensor([]).to(alex2.device)
    pred3 = Tensor([]).to(alex3.device)
    pred4 = Tensor([]).to(alex4.device)
    true = Tensor([]).to(alex1.device)
    loader = DataLoader(validation, batch_size=500, shuffle=False)
    for i, data in enumerate(loader, 0):
        x = data[0].to(alex1.device)
        y = data[1].to(alex1.device)
        pred1 = torch.cat((pred1, Function.label_to_36_argmax(alex1.predict(x), device=alex1.device)), dim=0)
        pred3 = torch.cat((pred3, Function.label_to_36_argmax(alex3.predict(x), device=alex3.device)), dim=0)
        true = torch.cat((true, y), dim=0)

    loader = DataLoader(validation_scaled, batch_size=500, shuffle=False)
    for i, data in enumerate(loader, 0):
        x = data[0].to(alex1.device)
        pred2 = torch.cat((pred2, Function.label_to_36_argmax(alex2.predict(x), device=alex1.device)), dim=0)
        pred4 = torch.cat((pred4, Function.label_to_36_argmax(alex4.predict(x), device=alex3.device)), dim=0)
    pred += pred1 + pred2*1.1 + pred3 + pred4



    data_size = len(pred)
    numbers = pred[:, :10]
    letters = pred[:, 10:]
    num_pred = torch.argmax(numbers, dim=1)
    letter_pred = torch.argmax(letters, dim=1) + 10

    output = torch.zeros((data_size, 36), dtype=torch.int, device=alex1.device)
    output[range(data_size), num_pred] = 1
    output[range(data_size), letter_pred] = 1
    print(accuracy(output, true))

In [ ]:
# obtain predictions on the test set
def run_committee_test():
    """
    shit code
    don't do anything to this
    :return:
    """
    training_set = read_train_labeled(DATASET_PATH)
    validation_proportion = 0.1
    validation, train = partition(training_set, [int(len(training_set) * validation_proportion)], False)
    train_scaled = preprocess_scale_image(train)
    train_scaled_relabeled = preprocess_260_labels(train_scaled)
    train_relabeled = preprocess_260_labels(train)

    validation = read_test(DATASET_PATH)
    validation_scaled = preprocess_scale_image(validation)
    validation_scaled_relabeled = preprocess_260_labels(validation_scaled)
    validation_relabeled = preprocess_260_labels(validation)


    p1 = {'n_way': 1, 'depth': (3, 4, 4), 'scaled': False, 'relabeled': False}
    p2 = {'n_way': 1, 'depth': (3, 4, 4), 'scaled': True, 'relabeled': False}
    p3 = {'n_way': 1, 'depth': (3, 4, 4), 'scaled': False, 'relabeled': True}
    p4 = {'n_way': 1, 'depth': (3, 4, 4), 'scaled': True, 'relabeled': True}

    original = Path(TRAINED_MODELS_PATH / 'alex-nets-rotation-augmented' / '1way-depth(3, 4, 4)')
    scaled = Path(TRAINED_MODELS_PATH / 'alex-nets-rotation-augmented-scaled' / '1way-depth(3, 4, 4)')
    relabeled = Path(TRAINED_MODELS_PATH / 'alex-nets-rotation-augmented-relabeled' / '1way-depth(3, 4, 4)')
    scaled_relabeled = Path(TRAINED_MODELS_PATH / 'alex-nets-rotation-augmented-scaled-relabeled' / '1way-depth(3, 4, 4)')

    alex1 = AlexNetClassifier(train, validation, **p1)
    alex2 = AlexNetClassifier(train_scaled, validation_scaled, **p2)
    alex3 = AlexNetClassifier(train_relabeled, validation_relabeled, **p3)
    alex4 = AlexNetClassifier(train_scaled_relabeled, validation_scaled_relabeled, **p4)
    alex1.load_network(original, 28)
    alex2.load_network(scaled, 30)
    alex3.load_network(relabeled, 7)
    alex4.load_network(scaled_relabeled, 28)
    pred1 = Tensor([]).to(alex1.device)
    pred2 = Tensor([]).to(alex2.device)
    pred3 = Tensor([]).to(alex3.device)
    pred4 = Tensor([]).to(alex4.device)
    loader = DataLoader(validation, batch_size=500, shuffle=False)
    for i, data in enumerate(loader, 0):
        x = data.to(alex1.device)
        pred1 = torch.cat((pred1, Function.label_to_36_argmax(alex1.predict(x), device=alex1.device)), dim=0)
        pred3 = torch.cat((pred3, Function.label_to_36_argmax(alex3.predict(x), device=alex3.device)), dim=0)

    loader = DataLoader(validation_scaled, batch_size=500, shuffle=False)
    for i, data in enumerate(loader, 0):
        x = data.to(alex1.device)
        pred2 = torch.cat((pred2, Function.label_to_36_argmax(alex2.predict(x), device=alex1.device)), dim=0)
        pred4 = torch.cat((pred4, Function.label_to_36_argmax(alex4.predict(x), device=alex3.device)), dim=0)
    pred = pred1*1.2 + pred2*1.1 + pred3 + pred4


    original = Path(TRAINED_MODELS_PATH / 'alex-nets' / '1way-depth(3, 4, 4)-fixmatch')
    scaled = Path(TRAINED_MODELS_PATH / 'alex-nets-scaled' / '1way-depth(3, 4, 4)-fixmatch')
    relabeled = Path(TRAINED_MODELS_PATH / 'alex-nets-relabeled' / '1way-depth(3, 4, 4)-fixmatch')
    scaled_relabeled = Path(
        TRAINED_MODELS_PATH / 'alex-nets-scaled-relabeled' / '1way-depth(3, 4, 4)-fixmatch')

    alex1 = AlexNetClassifier(train, validation, **p1)
    alex2 = AlexNetClassifier(train_scaled, validation_scaled, **p2)
    alex3 = AlexNetClassifier(train_relabeled, validation_relabeled, **p3)
    alex4 = AlexNetClassifier(train_scaled_relabeled, validation_scaled_relabeled, **p4)
    alex1.load_network(original, 26)
    alex2.load_network(scaled, 24)
    alex3.load_network(relabeled, 29)
    alex4.load_network(scaled_relabeled, 29)
    pred1 = Tensor([]).to(alex1.device)
    pred2 = Tensor([]).to(alex2.device)
    pred3 = Tensor([]).to(alex3.device)
    pred4 = Tensor([]).to(alex4.device)
    loader = DataLoader(validation, batch_size=500, shuffle=False)
    for i, data in enumerate(loader, 0):
        x = data.to(alex1.device)
        pred1 = torch.cat((pred1, Function.label_to_36_argmax(alex1.predict(x), device=alex1.device)), dim=0)
        pred3 = torch.cat((pred3, Function.label_to_36_argmax(alex3.predict(x), device=alex3.device)), dim=0)

    loader = DataLoader(validation_scaled, batch_size=500, shuffle=False)
    for i, data in enumerate(loader, 0):
        x = data.to(alex1.device)
        pred2 = torch.cat((pred2, Function.label_to_36_argmax(alex2.predict(x), device=alex1.device)), dim=0)
        pred4 = torch.cat((pred4, Function.label_to_36_argmax(alex4.predict(x), device=alex3.device)), dim=0)
    pred += pred1 + pred2*1.1 + pred3 + pred4




    data_size = len(pred)
    numbers = pred[:, :10]
    letters = pred[:, 10:]
    num_pred = torch.argmax(numbers, dim=1)
    letter_pred = torch.argmax(letters, dim=1) + 10

    output = torch.zeros((data_size, 36), dtype=torch.int, device=alex1.device)
    output[range(data_size), num_pred] = 1
    output[range(data_size), letter_pred] = 1
    # type cast to concatenated string
    pred = output
    pred = pred.detach().to('cpu').numpy().astype(int).astype(bytearray)
    result = []
    for i, row in enumerate(pred):
        result.append([i, ''.join(row.astype(str))])
    result = pd.DataFrame(result)
    result.columns = ["# ID", "Category"]

    # save predictions
    if not SUBMISSION_PATH.exists():
        SUBMISSION_PATH.mkdir(parents=True)
    result.to_csv(SUBMISSION_PATH / (str(15) + '.csv'), index=False)

In [ ]:
run_committee_val()
run_committee_test()